# Prediction de l'age de Mariage en Inde

## Project complet de Machine Learning avec Deploiement avec Flask-AWS

Model Building and  hosting local API Construction du model et creation de REST API
1. Data Preparation
2. Machine Learning Modelling (On utilisera Random Forest Regresor car nous faisons une prediction de type regression et non classification)
3.  Evaluation du Modele
4. Exportation du Trained Model
5. LOCAL REST API avec Flask web-server 
6. Creation d'un site webpour predire l'age de mariage en appellant le REST API 




In [1]:
import pandas as pd
data = pd.read_csv('age_of_marriage_data.csv')
print(data.shape)
data.head()

(2567, 10)


,id,gender,height,religion,caste,mother_tongue,profession,location,country,age_of_marriage
0,1,female,"5'4""",NaN,others,Telugu,NaN,London,United Kingdom,21.0
1,2,male,"5'7""",Jain,Shwetamber,Gujarati,Doctor / Healthcare Professional,Fairfax- VA,USA,32.0
2,3,male,"5'7""",Hindu,Brahmin,Hindi,Entrepreneurs / Business,Begusarai,India,32.0
3,4,female,"5'0""",Hindu,Thakur,Hindi,Architect,Mumbai,India,30.0
4,5,male,"5'5""",Christian,Born Again,Malayalam,Sales Professional / Marketing,Sulthan Bathery,India,30.0


# EDA

In [2]:
data.isnull().sum() #Verification des valeurs nulles dans chaque variables

id                   0
gender              29
height             118
religion           635
caste              142
mother_tongue      164
profession         330
location           155
country             16
age_of_marriage     19
dtype: int64

In [3]:
(data.shape[0] - data.dropna().shape[0])/data.shape[0] # (2567-1932)/2567
#data.shape[0] represente le nombre des lines dans le dataset
#0.24737047136735488 represente le pourcentage (24.7%) des donnees qui ont ete supprime dans le dataset
#2567 c'est le shape initial
#1932 c'est le shape qui nous reste en faisant le dropna

0.24737047136735488

In [4]:
data.dropna(inplace=True) #On supprime toutes les lignes qui ont des valeurs nulles

In [5]:
data.shape

(1932, 10)

In [6]:
data.head(60)

,id,gender,height,religion,caste,mother_tongue,profession,location,country,age_of_marriage
1,2,male,"5'7""",Jain,Shwetamber,Gujarati,Doctor / Healthcare Professional,Fairfax- VA,USA,32.0
2,3,male,"5'7""",Hindu,Brahmin,Hindi,Entrepreneurs / Business,Begusarai,India,32.0
3,4,female,"5'0""",Hindu,Thakur,Hindi,Architect,Mumbai,India,30.0
4,5,male,"5'5""",Christian,Born Again,Malayalam,Sales Professional / Marketing,Sulthan Bathery,India,30.0
5,6,male,"5'5""",Hindu,Valmiki,Hindi,Sportsman,Delhi,India,29.0
6,7,female,"5'2""",Hindu,Rajput - Lodhi,Hindi,Banking Professional,Jodhpur,India,28.0
7,8,male,"5'5""",Hindu,Bhatia,Punjabi,Entrepreneurs / Business,Faridabad,India,30.0
8,9,female,"5'5""",Jain,Shwetamber,Gujarati,Software Professional,Vadodara,India,35.0
9,10,female,"5'1""",Hindu,Billava,Tulu,HR Professional,Bengaluru / Bangalore,India,32.0
11,12,male,"5'9""",Hindu,Brahmin,Bengali,Finance Professional,Suri,India,32.0


In [7]:
data.profession.unique() #On check les valeurs de la variable profession

array(['Doctor / Healthcare Professional', 'Entrepreneurs / Business ',
       'Architect', 'Sales Professional / Marketing', 'Sportsman',
       'Banking Professional', 'Software Professional', 'HR Professional',
       'Finance Professional', 'Not Specified', 'Not working',
       'Chartered Accountant', 'Logistics and Travel Professional',
       'Defense Services', 'Team Member / Staff',
       'Managers and Senior Executives', 'Admin Professional',
       'Accounting Professional (Others)', 'Investment Professional',
       'Civil Engineer', 'Consultant / Supervisor / Team Leads',
       'Public Relations Professional', 'Training Professional (Others)',
       'Hotel & Hospitality Professional (Others)',
       'Software Professional (Others)', 'Nurse', 'Artist (Others)',
       'Non IT Engineer (Others)', 'Event Manager',
       'Marketing Professional', 'Science Professional (Others)',
       'Mechanical / Production Engineer', 'Research Assistant',
       'Electronics / Telecom

In [8]:
#Nous avons selectione seulement les colonnes qui nous servirons comme features et X contient nos features
X = data.loc[:,['gender','height','religion','caste','mother_tongue','country']]
#X = data.iloc[:,:-1] #Si nous voulons selectioner tout sauf la derniere colonne (Age) qui est notre target
y = data.age_of_marriage #Nous recuperons l'age comme target

In [9]:
X.head()

,gender,height,religion,caste,mother_tongue,country
1,male,"5'7""",Jain,Shwetamber,Gujarati,USA
2,male,"5'7""",Hindu,Brahmin,Hindi,India
3,female,"5'0""",Hindu,Thakur,Hindi,India
4,male,"5'5""",Christian,Born Again,Malayalam,India
5,male,"5'5""",Hindu,Valmiki,Hindi,India


In [10]:
from sklearn.preprocessing import LabelEncoder #Nous importons ceci pour encoder nos donnees en numerique
enc = LabelEncoder()
X.loc[:,['gender','religion','caste','mother_tongue','country']]= X.loc[:,['gender','religion','caste','mother_tongue','country']].apply(enc.fit_transform)
#Nous n'avons pas encode height (taille) car c'est en inche (' feet et '' inche) . On pourra le convertir d'abord en cm, pour un bon calcul

In [11]:
X.head()

,gender,height,religion,caste,mother_tongue,country
1,1,"5'7""",2,34,6,19
2,1,"5'7""",1,14,8,5
3,0,"5'0""",1,36,8,5
4,1,"5'5""",0,13,13,5
5,1,"5'5""",1,38,8,5


Explication de comment on va appliquer la conversion de pouce en cm/ On pourra maintenant creer une fonction qui fait cette conversion

In [12]:
#split('\'') represente la facon dont le pouce se presente ex: 5'7"
#5' represente 5 feet. Ainsi 1 feet= 30.48 cm
#7" represente 7". Ainsi 1 feet= 2.54 cm
#On teste pour le premier element  X.loc[1,'height'] qui est 5'7" et qui vaut 152.4 cm + 17.78 cm= 170.18cm
int(X.loc[1,'height'].split('\'')[0])*30.48

152.4

In [13]:
#On teste pour le premier element  X.loc[1,'height'] qui est 5'7" et qui vaut 152.4 cm
int(X.loc[1,'height'].split('\'')[1].replace('"',''))*2.54

17.78

In [14]:
#Cette fonction nous permet d'appliquer les calculs qui sont au-dessus
def h_cms(h):
    return int(h.split('\'')[0])*30.48 + int(h.split('\'')[1].replace('"',''))*2.54

In [15]:
#Nous appelons la methode h_cms en lui appliquant sur la colonne height et en creant une nouvelle colonne
X['height_cms'] = X.height.apply(h_cms)

In [16]:
X.head() #On a la nouvelle colonne qui contient les tailles converties en cm

,gender,height,religion,caste,mother_tongue,country,height_cms
1,1,"5'7""",2,34,6,19,170.18
2,1,"5'7""",1,14,8,5,170.18
3,0,"5'0""",1,36,8,5,152.40
4,1,"5'5""",0,13,13,5,165.10
5,1,"5'5""",1,38,8,5,165.10


In [17]:
X.drop('height',inplace=True,axis=1) #Nous supprimons la colonne height contenant les tailles en inche

In [18]:
X.head()

,gender,religion,caste,mother_tongue,country,height_cms
1,1,2,34,6,19,170.18
2,1,1,14,8,5,170.18
3,0,1,36,8,5,152.40
4,1,0,13,13,5,165.10
5,1,1,38,8,5,165.10


# Entrainement du modele

In [23]:
from sklearn.model_selection import train_test_split # On importe train_test_split pour diviser notre data set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0) #Test size on prend 30%

In [36]:
from sklearn.ensemble import RandomForestRegressor #Nous predirons en utilisant Random Forest Regresor 
model = RandomForestRegressor(n_estimators=80,max_depth=11) #On peut ajuster max_depth pour ajuster son modele
model.fit(X_train,y_train)
y_predict = model.predict(X_test)

# Evaluation du modele

In [37]:
from sklearn.metrics import mean_absolute_error, r2_score
print("MAE : ", mean_absolute_error(y_test,y_predict))
r2_score(y_test,y_predict)
#MAE une métrique qui mesure la performance/qualité mais pour évaluer la performance de votre modèle ML,
#une fois finalisé.
#Il ne faut pas le confondre avecMSE bien que les deux MAE/ MSE soient des mesures de performance,
#elles sont utilisées à deux étapes différentes d'un processus de modélisation et peuvent ne pas être liées. 
#Ainsi, bien qu'il soit logique d'évaluer votre modèle final sur MAE puisque vous serez jugé sur cette base,
#vous pouvez choisir l'un ou l'autre de MAE/ MSE comme critère (c'est-à-dire pour RandomForest) 
#en fonction de la performance à l'étape de validation.
#Ceci étant dit, gardez à l'esprit que vous pourriez vouloir évaluer les erreurs de validation 
#(c.-à-d. pour finaliser un modèle) sur la même métrique (c.-à-d. MAE dans ce cas), 
#pour garder la mesure d'erreur cohérente avec l'évaluation de l'ensemble de test.
#L'exemple ici est que notre modele predit les ages de mariage a une erreur d'une annee (MAE :  1.0441400442828088). 
#Ca veut dire que si le modele predit que vous aller vous marier en 2023, cela veut dire que cela peut etre en
#2022 ou 2023 ou 2024.
#r2_score nous permet d'evaluer la performance de notre modele. Et, ici c'est 68.5%, on pourra reajuster les parametres
#pour une bonne perfomance

MAE :  1.07419550332987


0.6850574766500873

# Export model

In [39]:
import joblib #On import joblib
joblib.dump(model,'ModelPredireAgeMariage.ml')
#Joblib nous permet de sauvegarder un modele ML pour enfin l'utiliser dans n'importe quelle circonstance ou environement
#Joblib est un ensemble d'outils pour fournir un pipelining léger en Python. Ainsi joblib.dump()nous permet de creer
#Une persistence

['ModelPredireAgeMariage.ml']

# Creation de l'API REST

On va maintenant cree un fichier app.py dans lequel nous allons construire notre application

#Dans un fichier qu'on va creer dans le meme repertoire que ce fichier jupyter notebook
#nomme app.py, on va mettre le code.
#Creation du fichier app.py


# Creation du site web

On va creer une page html dans lequel on mettra un formulaire qui prend les donnees sur la personne et qui appelle notre REST API